In [30]:
from cryptography.hazmat.primitives.asymmetric import rsa


In [24]:
import json
from uuid import UUID
from time import time as current_milli_time
from datetime import datetime
import hashlib

class Transaction:
    def __init__(self, from_id, to_id, amount, tr_id=None):
        self.from_id = from_id
        self.to_id = to_id
        self.tr_id = tr_id or str(UUID(int=0))  # Convert uuid4 to string if None
        self.amount = amount
        self.signature = None
    
    def __repr__(self):
        return json.dumps(
            {"transaction_id": str(self.tr_id), "from_id": self.from_id, "to_id": self.to_id, "amount": self.amount})

    def __hash__(self):
        return hashlib.sha256(str(self.__repr__()).encode()).hexdigest()

class Block:
    def __init__(self):
        self.timestamp = current_milli_time()
        self.transactions = []
        self.previous_hash = None
        self.balances = {}
        self.nonce = 0

    def to_json(self):
        return {
            "timestamp": self.timestamp,
            "transactions": [vars(tr) for tr in self.transactions],  # Serialize Transaction objects
            "previous_hash": self.previous_hash,
            "balances": self.balances,
            "nonce": self.nonce
        }
    
    @classmethod
    def from_json(cls, json_data):
        block = cls()
        block.timestamp = json_data["timestamp"]
        block.previous_hash = json_data["previous_hash"]
        block.balances = json_data["balances"]
        block.nonce = json_data["nonce"]
        # Deserialize Transaction objects
        for tr_data in json_data["transactions"]:
            tr = Transaction(tr_data["from_id"], tr_data["to_id"], tr_data["amount"], tr_data.get("tr_id"))
            block.transactions.append(tr)
        return block

    def __repr__(self):
        return json.dumps(self.to_json())

    def __hash__(self):
        return hashlib.sha256(str(self.__repr__()).encode()).hexdigest()


In [27]:
block = Block()
block.transactions.append(Transaction('alice', 'bob', 1))
block.__repr__()

'{"timestamp": 1715804402.023399, "transactions": [{"from_id": "alice", "to_id": "bob", "tr_id": "00000000-0000-0000-0000-000000000000", "amount": 1, "signature": null}], "previous_hash": null, "balances": {}, "nonce": 0}'

In [13]:
json.loads(json.loads(block.__repr__())['transactions'][0])

{'transaction_id': '054f11a9-bbb3-4770-888e-b4d84862d117',
 'from_id': 'alice',
 'to_id': 'bob',
 'amount': 1}

In [29]:
Block.from_json(json.loads("{\"timestamp\": 1715804402.023399, \"transactions\": [{\"from_id\": \"alice\", \"to_id\": \"bob\", \"tr_id\": \"00000000-0000-0000-0000-000000000000\", \"amount\": 1, \"signature\": null}], \"previous_hash\": null, \"balances\": {}, \"nonce\": 0}"))

{"timestamp": 1715804402.023399, "transactions": [{"from_id": "alice", "to_id": "bob", "tr_id": "00000000-0000-0000-0000-000000000000", "amount": 1, "signature": null}], "previous_hash": null, "balances": {}, "nonce": 0}